# Linear algebra in Julia
Based on work by Andreas Noack Jensen

## Basic linalg ops

First let's define a random matrix

In [1]:
A = rand(1:4,3,3)

3×3 Array{Int64,2}:
 1  3  2
 4  1  3
 2  1  1

Define a vector of ones

In [2]:
x = fill(1.0, (3))

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Notice that $A$ has type Array{Int64,2} but $x$ has type Array{Float64,1}. Julia defines the aliases Vector{Type}=Array{Type,1} and Matrix{Type}=Array{Type,2}. 

Many of the basic operations are the same as in other languages
#### Multiplication

In [3]:
b = A*x

3-element Array{Float64,1}:
 6.0
 8.0
 4.0

#### Transposition
As in other languages `A'` is the conjugate transpose

In [4]:
A'

3×3 LinearAlgebra.Adjoint{Int64,Array{Int64,2}}:
 1  4  2
 3  1  1
 2  3  1

and we can get the transpose with

In [5]:
transpose(A)

3×3 LinearAlgebra.Transpose{Int64,Array{Int64,2}}:
 1  4  2
 3  1  1
 2  3  1

#### Transposed multiplication
Julia allows us to write this without *

In [6]:
A'A

3×3 Array{Int64,2}:
 21   9  16
  9  11  10
 16  10  14

#### Solving linear systems 
The problem $Ax=b$ for ***square*** $A$ is solved by the \ function.

In [7]:
A\b

3-element Array{Float64,1}:
 1.0000000000000009
 1.0000000000000002
 0.9999999999999993

## Special Matrix Structures

Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system. Use the LinearAlgebra standard package to get access to structured matrices:

In [8]:
using LinearAlgebra

In [9]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [10]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [11]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-1.2318173450046868

In [12]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [13]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [14]:
@time eigvals(Asym);

  0.941590 seconds (781.73 k allocations: 45.239 MiB, 9.96% gc time)


In [15]:
@time eigvals(Asym_noisy);

  3.005748 seconds (18 allocations: 7.921 MiB)


In [16]:
@time eigvals(Asym_explicit);

  0.246433 seconds (7.61 k allocations: 8.378 MiB, 2.28% gc time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [17]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  1.584332 seconds (520.71 k allocations: 208.447 MiB, 6.94% gc time)


6.128348996321588